In [2]:
treshold = 0.45

# file = 'evaluation/multi_commits_rly_false_pairs_result.txt'
# file = 'evaluation/multi_commits_msr_result.txt'
# file = 'detection/multi_commits_second_nondup_part_newret.txt'

# file = 'detection/rly_false_pairs_newret.txt'
# file = 'detection/big_false_data_newret.txt'

file = 'detection/msr_multi_commits_no_repeat_newret.txt'
# file = 'detection/msr_multi_commits_no_repeat_newret5.txt'
# file = 'detection/multi_commits_second_nondup_part_newret.txt'
# file = 'detection/multi_commits_second_nondup_v_newret.txt'

with open(file) as f:
    ps = f.readlines()

second_r = ['elastic/elasticsearch', 'cocos2d/cocos2d-x', 'emberjs/ember.js', 'django/django', 'rails/rails', 'angular/angular.js', 'dotnet/corefx', 'ceph/ceph', 'ansible/ansible', 'hashicorp/terraform', 'joomla/joomla-cms', 'facebook/react', 'JuliaLang/julia', 'docker/docker']

pro_save = 0

save = []
for p in ps:
    p1, hist = p.strip()[:-1].split('[')
    r, n1, n2, ori, det, delta, zz = p1.split()
    
    
    if r not in second_r:
        continue
    
    
    if det == '2':
        continue

    hist = [float(h) for h in hist.split(',')]
    
    
    if len(hist) == 1:
        continue
    
    pro_save += 1
    
    s = -1
    for i in range(len(hist)):
        if hist[i] >= treshold:
            s = len(hist) - i - 1
            break
    save.append(s)
    
    '''
    if s > 0:
        print(r, n1, n2)
    '''
    
print('could_save', len(save))

print('all=', pro_save)

no_zero = 0
for i in save:
    if i > 0:
        no_zero += 1

no_neg = 0
for i in save:
    if i >= 0:
        no_neg += 1

print('no_zero=', no_zero / pro_save)
print('no_neg=', no_neg / pro_save)

save = list(filter(lambda x: x>= 0, save))
print(sum(save))
print(sum(save) / pro_save)


could_save 105
all= 105
no_zero= 0.8952380952380953
no_neg= 0.9238095238095239
260
2.4761904761904763


In [49]:
import sys
sys.path.append('/home/luyao/PR_get/INTRUDE')

from git import *

from datetime import datetime, timedelta

def checkt(repo, num1, num2):    
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    all_pa = get_pull_commit(p1)
    all_pb = get_pull_commit(p2)
    
    l_a, l_b = len(all_pa), len(all_pb)
    num_a, num_b = 0, 0
    now_a, now_b = None, None
    
    numt = 0
    while True:
        pa = all_pa[num_a] if num_a < l_a else None
        pb = all_pb[num_b] if num_b < l_b else None
        if (pa is None) and (pb is None):
            break
        
        if pa:
            pat = datetime.strptime(pa['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
        if pb:
            pbt = datetime.strptime(pb['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
        
        if (pb is None) or (pa and (pat < pbt)):
            num_a += 1
            now_a = pa
        else:
            num_b += 1
            now_b = pb
        
        if now_a and now_b:
            if (num_a < l_a) or (num_b < l_b):
                numt += 1
    
    if numt > 0:
        return True
    else:
        return False
    

with open('data/clf/second_nondup.txt') as f:
    pairs = f.readlines()

w = 0

out = open('data/multi_commits_second_nondup.txt', 'w')

pairs = sorted(pairs, key=lambda x: x.split()[0])
last_repo = None

for pair in pairs:
    repo, num1, num2 = pair.strip().split()
    
    #print(repo, num1, num2)
    '''
    if repo != last_repo:
        last_repo = repo
        init_model_with_repo(repo)
    '''
    
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    cl1 = get_pull_commit(p1)
    cl2 = get_pull_commit(p2)
    
    
    #t = get_pr_sim_vector(p1, p2)
    #o = m.predict_proba([t])[0][1]
    
    if (len(cl1) == 1) and (len(cl2) == 1):
        continue
    
    if (len(cl1) == 0) or (len(cl2) == 0):
        continue
    
    if (len(cl1) > 100) or (len(cl2) > 100):
        continue

    if check_large(p1) or check_large(p2):
        continue
    
    if not checkt(repo, num1, num2):
        continue

    print(pair.strip(), file=out)
    out.flush()

    w += 1
    if w % 100 == 0:
        print(w)

out.close()
print(w)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100


KeyboardInterrupt: 

In [1]:
r2 = set()
with open('data/clf/second_msr_pairs_nolarge.txt') as f:
	for t in f.readlines():
		r, n1, n2 = t.split()
		r2.add(r)

print(r2)


{'angular/angular.js', 'facebook/react', 'rails/rails', 'hashicorp/terraform', 'joomla/joomla-cms', 'docker/docker', 'ceph/ceph', 'dotnet/corefx', 'django/django', 'cocos2d/cocos2d-x', 'ansible/ansible', 'JuliaLang/julia', 'elastic/elasticsearch', 'emberjs/ember.js'}


In [21]:
from git import *

repo = 'rails/rails'
file = 'evaluation/rails_rails_select.txt'
with open(file) as f:
    for n in f.readlines():
        p = get_pull(repo, n.strip())
        if check_large(p):
            print(n)


In [42]:
treshold = 0.51

# file = 'evaluation/multi_commits_rly_false_pairs_result.txt'
# file = 'evaluation/multi_commits_msr_result.txt'

file = 'detection/multi_commits_second_nondup_part_newret.txt'

# file = 'detection/rly_false_pairs_newret.txt'
# file = 'detection/msr_multi_commits_no_repeat_newret.txt'
# file = 'detection/big_false_data_newret.txt'

with open(file) as f:
    ps = f.readlines()

# second_r = ['elastic/elasticsearch', 'cocos2d/cocos2d-x', 'emberjs/ember.js', 'django/django', 'rails/rails', 'angular/angular.js', 'dotnet/corefx', 'ceph/ceph', 'ansible/ansible', 'hashicorp/terraform', 'joomla/joomla-cms', 'facebook/react', 'JuliaLang/julia', 'docker/docker']

save = []
for p in ps:
    p1, hist = p.strip()[:-1].split('[')
    r, n1, n2, ori, det, delta, zz = p1.split()
    
    '''
    if r not in second_r:
        continue
    '''
    
    if det == '2':
        continue

    hist = [float(h) for h in hist.split(',')]
        
    
    if len(hist) == 1:
        continue
    

    s = -1
    for i in range(len(hist)):
        if hist[i] >= treshold:
            s = len(hist) - i - 1
            break
    save.append(s)
    
    
    if s >= 0:
        print(r, n1, n2, len(hist))
    
    
print('all', len(save))

no_fuyi = 0
for i in save:
    if i >= 0:
        no_fuyi += 1

print('no_zero=', no_fuyi / len(save))


angular/angular.js 15836 5385 2
all 279
no_zero= 0.0035842293906810036


In [47]:
import git
from datetime import datetime, timedelta


def checkt(repo, num1, num2):    
    p1 = git.get_pull(repo, num1)
    p2 = git.get_pull(repo, num2)
    
    all_pa = git.get_pull_commit(p1)
    all_pb = git.get_pull_commit(p2)
    
    l_a, l_b = len(all_pa), len(all_pb)
    num_a, num_b = 0, 0
    now_a, now_b = None, None
    
    numt = 0
    while True:
        pa = all_pa[num_a] if num_a < l_a else None
        pb = all_pb[num_b] if num_b < l_b else None
        if (pa is None) and (pb is None):
            break
        
        if pa:
            pat = datetime.strptime(pa['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
        if pb:
            pbt = datetime.strptime(pb['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
        
        if (pb is None) or (pa and (pat < pbt)):
            num_a += 1
            now_a = pa
        else:
            num_b += 1
            now_b = pb
        
        if now_a and now_b:
            if (num_a < l_a) or (num_b < l_b):
                numt += 1
    
    if numt > 0:
        return True
    else:
        return False

num, tot = 0, 0
with open('data/clf/second_msr_pairs_nolarge.txt') as f:
    for t in f.readlines():
        r, n1, n2 = t.split()
        
        tot += 1
        if checkt(r, n1, n2):
            num += 1
        
        if tot % 100 == 0:
            print(1.0 * num / tot)

print(num)
print(1.0 * num / tot)


0.27
0.25
0.2966666666666667
0.34
0.324
0.31833333333333336
0.3142857142857143
0.3425
0.3511111111111111
0.327
333
0.31563981042654027


In [6]:
from git import *

with open('data/clf/second_nondup.txt') as f:
    pairs = f.readlines()

pairs = sorted(pairs, key=lambda x: x.split()[0])

num = 0

for pair in pairs:
    repo, num1, num2 = pair.strip().split()
        
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    cl1 = get_pull_commit(p1)
    cl2 = get_pull_commit(p2)
    
    if (len(cl1) == 1) and (len(cl2) == 1):
        num += 1
    
    if num % 100 == 0:
        print(num)

print('num=', num)

0
100
100
200
200
300
400
500
500
500
500
600
600
600
700
800
900
1000
1000
1000
1000
1100
1200
1200
1200
1200
1300
1300
1400
1400
1400
1400
1500
1600
1600
1600
1600
1600
1600
1600
1700
1700
1800
1800
1800
1800
1900
1900
2000
2000
2000
2100
2100
2200
2200
2300
2400
2400
2500
2500
2600
2700
2700
2800
2800
2800
2900
3000
3100
3100
3200
3200
3300
3300
3400
3500
3500
3500
3500
3500
3600
3600
3700
3700
3700
3700
3700
3800
3900
4000
4100
4100
4100
4100
4100
4100
4200
4200
4300
4300
4400
4500
4600
4700
4700
4800
4800
4900
4900
4900
4900
5000
5100
5100
5100
5200
5200
5200
5300
5400
5500
5600
5700
5700
5800
5900
5900
6000
6000
6100
6200
6300
6400
6500
6500
6600
6700
6700
6700
6800
6800
6900
6900
6900
6900
7000
7000
7000
7000
7100
7200
7300
7300
7400
7400
7500
7600
7700
7800
7800
7900
7900
7900
8000
8000
8100
8200
8200
8300
8300
8400
8400
8500
8600
8700
8800
8800
8900
9000
9100
9200
9300
9400
9500
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11

finish write /DATA/luyao/pr_data/facebook/react/5953/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3467/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6362/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/12635/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/748/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/5609/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/7920/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/5199/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/11557/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6860/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/2068/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/107/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/4477/commits.jso

finish write /DATA/luyao/pr_data/facebook/react/5559/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/13620/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/1094/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/12537/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6576/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/10039/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8285/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/2418/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9111/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/12909/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9281/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9049/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/4584/commits

finish write /DATA/luyao/pr_data/facebook/react/865/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9659/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8780/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/4750/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/11662/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/12833/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6123/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/12746/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3425/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/10943/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8096/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/5877/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/2727/commits.

finish write /DATA/luyao/pr_data/facebook/react/12385/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6381/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/207/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/10561/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/5038/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9139/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6257/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/4780/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/10560/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8269/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9185/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/5721/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8899/commits.j

...

finish write /DATA/luyao/pr_data/facebook/react/8465/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/4332/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6963/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9153/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/7532/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3540/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3016/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/1952/commits.json to file....
21900
finish write /DATA/luyao/pr_data/facebook/react/7813/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/11654/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8298/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8742/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9288/comm

finish write /DATA/luyao/pr_data/facebook/react/4089/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/11437/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8156/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3651/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/11439/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/10771/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/6763/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9287/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/4944/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/9780/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/11790/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/7133/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3047/commits

finish write /DATA/luyao/pr_data/facebook/react/3464/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/7873/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/199/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/7596/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8383/commits.json to file....
22400
22400
finish write /DATA/luyao/pr_data/facebook/react/13157/commits.json to file....
22400
finish write /DATA/luyao/pr_data/facebook/react/7102/commits.json to file....
22400
finish write /DATA/luyao/pr_data/facebook/react/4975/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/3263/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/878/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8082/commits.json to file....
finish write /DATA/luyao/pr_data/facebook/react/8029/commits.json to file....
finish write /DATA/luyao/pr_data/facebook

finish write /DATA/luyao/pr_data/hashicorp/terraform/10383/commits.json to file....
finish write /DATA/luyao/pr_data/hashicorp/terraform/13982/commits.json to file....


GitHubError: 403: API rate limit exceeded for user ID 7541296.